# 데이터 불러오기 및 전처리

In [1]:
!pip install implicit

  Using cached implicit-0.4.4.tar.gz (1.1 MB)
  Created wheel for implicit: filename=implicit-0.4.4-cp38-cp38-win_amd64.whl size=614968 sha256=cf160261daee9838fc1e8605689b48173168268ea864a8a1ebed0175b46852c4
  Stored in directory: c:\users\pc\appdata\local\pip\cache\wheels\00\ac\67\6f4536c819ed560c2c7e17c0f7a920e3e50c26108616087d05
Successfully built implicit


In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')
import re
import scipy 

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error

from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer


In [3]:
from implicit.evaluation import  *
from implicit.als import AlternatingLeastSquares as ALS
from implicit.bpr import BayesianPersonalizedRanking as BPR

# 컨텐츠 기반 데이터 사용 ( TFIDF , W2V )

## -데이터 불러오기 및 전처리

In [5]:
upper1=pd.read_excel(r'C:\Users\PC\프로젝트\컨텐츠데이터\상의1.xlsx',thousands = ',')
upper2=pd.read_excel(r'C:\Users\PC\프로젝트\컨텐츠데이터\상의2.xlsx',thousands = ',')
upper3=pd.read_excel(r'C:\Users\PC\프로젝트\컨텐츠데이터\하의1.xlsx',thousands = ',')
upper4=pd.read_excel(r'C:\Users\PC\프로젝트\컨텐츠데이터\하의2.xlsx',thousands = ',')

def remove(a):
    a.price=a.price.str.replace('\n','')
    a.tag=a.tag.str.replace('\n','')
    a.replace('Null',np.nan,inplace=True)
    return a


# 모든 컨텐츠데이터 불러온 후 하나의 데이터프레임으로 만들기
df=pd.concat([upper1,upper2,upper3,upper4]).reset_index()
remove(df)
DF=df[df['tag'].notnull()]
DF.reset_index(inplace=True)
DF.drop(['level_0','index'],axis=1,inplace=True)
DF.tag=DF.tag.str.split('#')
for i in range(DF.shape[0]):
    del DF.tag[i][0]
DF['item_name']=DF.item_name.str.replace(r'\[[^)]*\]', '')
DF['item_name']=DF.item_name.str.replace(r'\([^)]*\)', '')
DF=DF.drop_duplicates(['item_name'])
DF=DF.astype({'score_average':'float'})
DF=DF.astype({'review_count':'float'})
DF.describe()

,score_average,review_count,like_count
count,1945.000000,1646.000000,2625.000000
mean,4.714859,4249.758809,3219.206476
std,0.250247,7804.381531,6434.393090
min,2.000000,10.000000,0.000000
25%,4.700000,290.000000,29.000000
50%,4.700000,1436.500000,758.000000
75%,4.800000,4659.750000,3710.000000
max,5.000000,104967.000000,72628.000000


## -가중 평균


- r : 개별 옷에 평점을 준 횟수 
- m : 평점을 부여하기 위한 최소 리뷰 개수 
- R : 개별 영화에 대한 평균평점
- M : 전체 영화에 대한 평균평점

In [6]:
# 가중평균
def weighted_vote_average(record):
    r=record['review_count']
    R=record['score_average']
    return ( (r/(r+m)*R) ) + ( (m/(m+r)*M)  )

In [7]:
M=np.round(DF['score_average'].mean(),4)
m=DF['review_count'].quantile(0.5)
DF['weighted_vote']=DF.apply(weighted_vote_average,axis=1)

## 1. TFIDF

In [8]:
DF.tag_tfidf=DF.tag.apply(lambda x : (' ').join(x))
DF['tag_tfidf']=DF.tag_tfidf

tfidf=TfidfVectorizer(min_df=1,ngram_range=(1,3))
tag_tf=tfidf.fit_transform(DF['tag_tfidf'])

### -유사도 함수

In [9]:
# 1. 코사인유사도
from sklearn.metrics.pairwise import cosine_similarity
tf_sim=cosine_similarity(tag_tf,tag_tf)
# 2. 유클리디언 유사도
from sklearn.metrics.pairwise import euclidean_distances
tf_euc=euclidean_distances(tag_tf,tag_tf)
# 3. 맨하탄 유사도 
from sklearn.metrics.pairwise import manhattan_distances
tf_man=manhattan_distances(tag_tf,tag_tf)

In [10]:
#최종 알고리즘
def sim_clothes(clothe,sim,top_n):
    
    # 찾고자 하는 옷선택
    clothes=DF[DF.item_name==clothe]
    # 찾고자 하는 옷의 인덱스 값만 추출하여 새로운 변수에 저장
    clothes_index=clothes.index.values
    
    # 코사인유사도를 구한 행렬을 역순으로 정렬 -> 유사도가 높은 순의 인덱스로 정렬됨 
    sim_sorted_ind=sim.argsort()[:,::-1]
    
    # 내가 추출하려는 옷의 인덱스의 행번째추출, 이중리스트이므로 [0][0]
    # -> 아이템이름을 추출 -> 내가 원하는 옷의 이름과 같지 않다면
    if DF.iloc[sim_sorted_ind[clothes.index][0][0]].item_name != clothe:
        
        # 내가 추출하려는 옷의 인덱스 행번째의 array를 리스트로 변경, 변수명 l로 저장
        l=sim_sorted_ind[clothes.index][0].tolist()
        # 내가 추출하려는 옷의 인덱스 값 자체를 리스트로 변경한 후 값으로 추출
        v=clothes.index.values.tolist()[0]
        
        # 리스트 l 에서 내가 추출하려고 한 옷의 값을 제거
        # -> 내가 추출하고자 하는 옷의 인덱스 번호가 제거된 리스트 생성
        l.remove(clothes.index.values.tolist()[0])
        
        # 리스트l의 array형태로 맞춰주기
        sim_sorted_ind2=np.array(l)
        
    
        # 자기 자신을 제외하고 유사도가 높은 인덱스를 top_n개수 만큼 추출
        # 내가 추출하고자 하는 인덱스의 값이 제거된 상태이므로 0값부터 원하는 개수만큼 출력
        similar_indexes=sim_sorted_ind2[0:(top_n)]
        a=  DF.iloc[similar_indexes].sort_values(by='weighted_vote',ascending=False).drop('tag_tfidf',axis=1)
    
    else :
        # sim_indexes는 이중리스트 상태이므로 이중리스트를 해제한 후 인덱스를 이용해 해당 내용 추출
        similar_indexes=sim_sorted_ind[clothes_index,1:(top_n+1)]
        a= DF.iloc[similar_indexes[0]].sort_values(by='weighted_vote',ascending=False).drop('tag_tfidf',axis=1)
    
    return a

### -sim_clothes 로 구현

In [11]:
sim_clothes('릴렉스 핏 크루 넥 반팔 티셔츠 ',tf_sim,5)

,item_name,brand,score_average,review_count,like_count,tag,price,weighted_vote
252,베이식 긴팔 티셔츠,MUSINSASTANDARD,4.8,94424.0,35656.0,"[기본티, 긴팔티, 라운드넥, 루즈핏, 면티셔츠, 무지티, MU-T, 홈웨어컬렉션]","13,900원",4.798725
77,실켓 릴렉스 핏 크루 넥 반팔 티셔츠,MUSINSASTANDARD,4.8,398.0,1302.0,"[기본티, 라운드넥, 루즈핏, 면티셔츠, 무지티, MU-T, 프리미엄컬렉션, 유아인...","30,900원",4.733363
120,크루 넥 7부 티셔츠,MUSINSASTANDARD,4.7,7820.0,3312.0,"[기본티, 라운드넥, 루즈핏, 면티셔츠, 무지티, 반팔티, MU-T, 홈웨어컬렉션]","12,900원",4.702312
186,릴렉스 핏 크루 넥 포켓 반팔 티셔츠,MUSINSASTANDARD,4.7,8518.0,938.0,"[기본티, 라운드넥, 루즈핏, 면티셔츠, 무지티, 반팔티, MU-T, 홈웨어컬렉션]","12,900원",4.702150
7,베이식 크루 넥 반팔 티셔츠,MUSINSASTANDARD,4.7,24975.0,10044.0,"[기본티, 라운드넥, 면티셔츠, 무지티, 반팔티, MU-T, 홈웨어컬렉션]","10,900원",4.700810


## Word2vec

In [12]:
word=[]
for words in DF.tag:
    word.append(words)


- W2V 함수. Hyper Parameter 조정 가능

In [13]:
word2vec=Word2Vec(sentences=word,window=5,sg=1,min_count=1,vector_size=100,epochs=100)

In [14]:
# 단어 벡터 평균구하기
def vectors(embedding):
    word_embedding=[]
    dec2vec=None
    count=0
    for word in embedding:
        try : 
            # word에 해당하는 단어를 워드투백에서 찾아 리스트에 붙힘
            # 100차원으로 형성됨
            word_embedding.append(word2vec.wv[word]) 
            count+=1
        except : continue
    #print('='*30,"word_embedding1","="*30)
    #print(word_embedding)
    
    word_embedding2=np.array(word_embedding)
    #print('='*30,"word_embedding2","="*30)
    #print(word_embedding2)
    
    v=word_embedding2.sum(axis=0)  
    print(v)
    
    if type(v)!=np.ndarray:
        return np.zeors(100)
    
    return(v/count)

In [15]:
wordvec=[vectors(x) for x in word]

[ 2.3671727   3.8107731   3.6210818   5.011609   -3.4920418   2.4348478
 -6.396918    5.063914   -5.0189056   0.6119392   0.9191912  -9.615893
 -3.54033     6.1087036   1.1743356   3.1670256   5.8019743  -2.1792936
 -1.6937685  -4.7741675  -0.34360105 -5.0682244  -6.8591633  -0.8379164
 -5.5979815   4.2106647   1.4300045   0.49906343 -9.120501    2.247984
 -1.3023777   4.2546453  -4.552772   -2.9262173  -6.0925894   0.7369018
  4.25055    -4.709629   -2.7978632  -6.7384205   0.01928002 -2.3769186
 -3.8495622   2.2924016   4.077151   -4.268137    2.9035635   3.1156933
  1.4525199   1.2922876   0.6589445   3.4790847   9.561407   -1.1823901
  0.09360057 -4.3006454  -3.8525872  -3.865404    2.2541337  -6.133554
  9.219411   -6.834716    0.8708428  -4.5694046  -3.2194648  -0.22666812
  1.13062     4.6088476  -2.9796019   5.368741    4.6769433  -1.1968745
  3.821116    0.8193743   3.8895693   7.9754076   1.0550135   0.6014351
 -2.5870907  -0.99278617 -1.1165214  -1.323876    0.32687637  3.77

  3.6951869  -1.4120038  -0.9696593  -2.3069694 ]
[ 3.31516385e-01  6.97381854e-01  1.65784287e+00  5.30974960e+00
 -1.08896279e+00 -4.72712144e-03 -2.48184800e+00  3.13996643e-01
 -1.23510396e+00  1.53722227e-01  8.50263953e-01 -3.42297888e+00
 -1.88848138e+00  1.33290803e+00  1.39069855e+00  5.38822711e-01
  3.58822989e+00 -1.22151136e+00  4.93821323e-01 -2.24538803e+00
 -1.48290908e+00 -2.77302098e+00 -2.91056848e+00  2.75595903e-01
 -2.52532530e+00  4.33231449e+00  4.27242398e-01  3.12573910e+00
 -5.34179354e+00 -1.21754825e-01  6.76257133e-01  1.36021018e+00
 -2.32360101e+00 -2.08964157e+00 -2.41813231e+00 -1.67201233e+00
  1.51446712e+00 -2.36035013e+00 -2.18624544e+00 -2.01655364e+00
 -2.38715336e-01 -1.89626181e+00 -4.05443335e+00  9.08530533e-01
  2.83461547e+00 -2.56971240e+00  1.75753355e+00  1.63987219e+00
  2.60966611e+00  3.93949151e-01  1.10547018e+00  2.78327370e+00
  3.49958563e+00  6.55035749e-02 -1.90198863e+00 -1.64982939e+00
 -3.31825376e+00 -3.33463526e+00  5.2977

[ 2.293859    2.0530145   3.1737342  -0.18329109  2.9008956  -0.71739197
 -2.699979    6.533132    0.54071087 -0.7100327   3.52867    -5.2205853
 -0.45663166  4.364383    3.8134131  -0.07479236  3.9655967  -4.807603
 -3.6361423  -1.8680371  -2.3013835  -5.186245   -2.859644    0.36231577
  1.6306334   2.083455    3.5560093  -0.27552944 -3.6630154  -2.2405782
 -0.21924195  1.5733051   1.8015435  -4.871916   -1.0911486  -2.1963437
  2.330673   -0.9463488  -1.1841664  -5.5680404  -3.0030339  -5.544867
  0.58589506 -1.4725169  -2.2655714  -2.6899574   0.7514146   3.128123
 -0.4318966   3.5188231   1.0935001   5.648504   -2.6697652   1.7849269
 -1.1184341  -0.7136096   0.8777735  -3.8116179  -0.74969757  1.621266
  3.84464    -8.421134   -0.9099129   3.7234738  -1.4379781   2.5025187
  1.3655671  -0.2265608  -1.2046802   5.1383166  -1.8603628   0.6957639
 -3.4968061  -0.13626617  3.5599093   3.1662064  -1.6785553   3.6602325
  2.0872195   0.48855555  3.024891    3.1581903  -2.7429264  -1.62

[ 0.70148003  0.9995927   1.0231352   0.6776774  -1.3632128  -0.27007467
 -0.14215285  2.3527875  -1.7205813  -1.6617427   0.7408949  -4.5857754
 -2.7749653   3.0605378  -0.83617204  0.2515739   2.8855784  -0.82924384
 -2.6399145  -3.329218   -0.79416287 -2.2053628  -0.77886295  2.163381
 -0.15066569  0.5580742  -0.41696385 -0.732365   -2.9484332   2.2149215
  0.98063743  0.21189696 -0.2916828   0.03026676 -0.9304286   3.1258123
  1.5350425  -0.12815756 -0.91209906 -1.4594723  -2.9869025   0.18285987
  1.1309855   2.2015758   0.8352212  -3.8594189   1.2506523   1.9122069
  1.0886673   1.9203162   0.19020012  0.92267084  1.8970006  -0.827488
 -1.424655   -2.172276   -0.8032068   1.3991535   1.634753   -0.73722386
  2.8200867  -3.7876158  -1.2910659  -1.026645   -3.214353    1.0734195
  1.575163    2.5716743   2.4706059  -0.4956835   1.4477326   0.35661876
  1.8579226  -0.5948918  -0.25448102  4.9914317   1.2726011   1.2345952
 -0.5964934  -0.83766574 -0.5153372  -2.9700055   1.1545305  

  1.695373   -0.18664981 -2.8396645  -3.901473  ]
[ 3.1593287e+00 -4.9628335e-01  2.4305201e+00  7.8582630e+00
 -5.0922203e+00 -2.4994221e+00 -4.6445780e+00  2.0497069e+00
 -2.3735151e+00 -2.2315452e+00  1.5271921e-01 -1.1218287e+00
 -3.1821914e+00  3.9088404e+00  1.9300508e+00  9.8286760e-01
  3.4457061e+00 -2.2217166e+00  5.9676528e-01 -2.6760714e+00
 -1.6867468e+00 -3.2330134e+00 -2.4240336e+00 -3.8684148e-01
 -1.2512519e+00  1.0779260e+00  1.1822017e+00  5.3329344e+00
 -6.6549191e+00 -6.8188435e-01  3.0489180e+00  1.7680107e+00
  2.2070084e+00 -2.3585122e+00 -2.1832106e-01 -1.3713531e+00
 -3.8075873e-01 -5.8639340e+00 -5.8070707e+00 -1.8670688e+00
 -2.8203125e+00 -3.1682954e+00 -2.4565637e+00  1.1691153e+00
  1.0504540e+00 -4.2711334e+00  1.1570544e+00  1.9363950e+00
  4.2168036e+00  4.2777987e+00  2.4082680e+00  3.8629594e+00
  3.0122643e+00 -1.3109413e+00 -1.5997568e+00 -1.8598772e+00
 -3.7702179e+00 -2.9596155e+00  7.1568155e-01 -4.2315607e+00
  3.1675694e+00 -1.4171842e-01 -7.9

...

[ 4.1310449e+00  4.4339797e-01  3.4823232e+00  9.3888626e+00
 -3.5753736e+00 -1.6771061e+00 -4.3187690e+00  1.4853803e+00
 -1.8152766e+00 -1.6190009e+00  6.1242324e-01 -1.1347829e+00
 -1.1300184e+00  2.8796821e+00  3.4961121e+00  7.4010539e-01
  3.0936453e+00 -1.6792265e+00 -1.0075177e+00 -3.2754340e+00
 -1.6085286e+00 -3.5306275e+00 -3.4411418e+00 -6.8406367e-01
 -1.0893408e+00  1.1044693e+00 -7.6573312e-01  5.3651719e+00
 -6.5930219e+00 -7.2905421e-04  1.7734965e+00  4.1289240e-02
  1.5406319e-01 -2.0554378e+00 -1.8362125e+00 -8.9675075e-01
 -4.3432757e-01 -4.4657450e+00 -5.9737701e+00 -1.9369946e+00
 -1.8704870e+00 -1.6598516e+00 -3.3815720e+00  7.3440206e-01
  8.1147909e-01 -3.6598790e+00  2.1770222e+00  1.2368543e+00
  3.8178434e+00  4.8380237e+00  2.6441658e+00  2.6946480e+00
  4.0378828e+00 -4.2331865e-01 -1.8495524e+00 -6.5257555e-01
 -4.2078342e+00 -2.6920092e+00  1.8164701e+00 -3.6059065e+00
  2.6991792e+00  4.2022836e-01  1.1107412e-01 -1.0110132e+00
 -2.9896815e+00  2.77314

 -1.07359953e-01  1.90171469e-02 -7.21496165e-01  3.57821472e-02]
[ 0.1714747   1.8793802   0.9001075   0.5763103  -3.9485583  -2.4132977
  0.19835147  5.7756743  -1.5145683  -3.523395    0.6803598  -3.481478
 -1.5817909   1.8559986  -2.0487924  -3.09711     3.1331847  -2.781552
  0.13251218 -5.7331257   1.8869685  -3.654948    1.2560639   1.2023498
 -1.9729034   2.4586256  -0.8892281   1.4234954  -4.653747    2.9197097
  1.4278916   1.2442925   1.6426803  -2.431435    0.39355785  0.5751188
 -0.36376232 -1.4594214  -0.37627667 -3.3001144  -0.53317237 -3.960792
 -2.324658    1.3138531   2.4961474  -7.160474    0.81996965  1.946853
  1.5439751   0.44026184  2.3268101   1.6667721   2.956659   -2.463459
 -0.9683542  -2.9154463  -0.5523312  -1.4621533  -0.55283886  1.4026364
  4.4270706  -4.7121224   0.6939875  -2.3673759  -4.966198    4.2463026
  4.495121    1.1015863   1.3179697   1.2692852   0.62776625 -1.0184174
  1.5423388   0.5914749   2.0263758   4.442911    1.718965    0.46526018
  

  1.3168039  -0.48713166 -2.8396375  -4.1524186 ]
[ 0.75053596  5.1143856   4.584702   -2.4560397  -3.4466214  -2.1261795
 -1.4289887   4.8507485  -4.2138076  -4.4521117   4.589512   -3.6442425
  3.480252   -0.37181008  0.9274012   0.02828714  1.9043126   0.3004962
 -4.433015   -1.8999647  -2.41262    -6.4385915  -0.636105    3.3655465
 -3.2495084   1.037737   -1.1147974  -1.0150325   1.016121    0.97575563
  4.122417    2.153246    2.4592323   0.47126552  2.5132658  -0.1745965
  3.5886354  -1.9465454   0.7461164  -3.6713016  -1.4530061  -2.2969584
 -0.3827964  -1.582153    0.65055895 -2.9772441   4.1646714  -0.4251116
 -2.080257   -0.12871705  2.5394087   0.7379812   0.72723913 -1.7367655
  1.7522273  -2.4373999   1.8517158   0.39534658  1.8399341   1.3912058
 -0.05061428  0.93384725 -0.5598922  -0.45288548 -3.0008268   1.9257479
  1.134756    0.07899913  0.57494843  3.191679    2.4417996  -2.7475057
  0.6713537   0.04459789 -1.5086694   5.3325534   0.35915253  0.6112411
  4.5755305  

[-2.3850336   5.899974    1.1398547   0.5138141  -0.78802013 -0.5977013
 -0.6796459   2.6108727  -8.169187   -2.756843    4.1925445  -5.450325
  0.9672017   1.6036917   3.0649903  -1.2748619   5.7915173  -1.4206171
 -6.1153126  -1.281355   -1.040642   -1.2875459   5.935462    1.8730363
 -3.3050044  -1.5511322  -4.9304895  -0.15676624  0.4878393  -1.4440857
 -0.13710627 -1.2527843  -0.30108458 -0.30474842 -0.4160419   2.6940825
 -1.7597276  -2.8901126  -2.4236794  -4.4154935  -5.0519996  -1.2046766
 -0.19223723  0.42319474  2.6471248   1.6870126  -1.6674109  -3.976551
 -0.36029106  1.1112169   1.2138704  -0.0983445   2.797995   -3.9860241
 -1.5907092  -6.2554784   0.06161173 -0.77216417 -1.7088696  -0.60605186
  0.21647155 -2.9488986   1.8804736  -2.258763   -4.392329    4.0196023
 -1.4283478   2.810955   -3.2680087   5.480594    0.86736774 -0.3879472
  3.5370986   0.47362894 -2.744823    3.7439957   1.3455487  -1.4507989
  1.2959824  -4.2200155  -2.6707761  -2.281571   -0.7860497   0.8

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### -유사도함수

In [16]:
# 1. 코사인유사도
w2v_sim=cosine_similarity(wordvec,wordvec)
# 2. 유클리디언 유사도
w2v_euc=euclidean_distances(wordvec,wordvec)
# 3. 맨하탄 유사도 
w2v_man=manhattan_distances(wordvec,wordvec)

In [17]:
sim_clothes('릴렉스 핏 크루 넥 반팔 티셔츠 ',w2v_sim,5)

,item_name,brand,score_average,review_count,like_count,tag,price,weighted_vote
77,실켓 릴렉스 핏 크루 넥 반팔 티셔츠,MUSINSASTANDARD,4.8,398.0,1302.0,"[기본티, 라운드넥, 루즈핏, 면티셔츠, 무지티, MU-T, 프리미엄컬렉션, 유아인...","30,900원",4.733363
120,크루 넥 7부 티셔츠,MUSINSASTANDARD,4.7,7820.0,3312.0,"[기본티, 라운드넥, 루즈핏, 면티셔츠, 무지티, 반팔티, MU-T, 홈웨어컬렉션]","12,900원",4.702312
186,릴렉스 핏 크루 넥 포켓 반팔 티셔츠,MUSINSASTANDARD,4.7,8518.0,938.0,"[기본티, 라운드넥, 루즈핏, 면티셔츠, 무지티, 반팔티, MU-T, 홈웨어컬렉션]","12,900원",4.702150
7,베이식 크루 넥 반팔 티셔츠,MUSINSASTANDARD,4.7,24975.0,10044.0,"[기본티, 라운드넥, 면티셔츠, 무지티, 반팔티, MU-T, 홈웨어컬렉션]","10,900원",4.700810
262,크루 넥 긴팔 티셔츠,MUSINSASTANDARD,4.7,28360.0,5463.0,"[기본티, 긴팔티, 라운드넥, 루즈핏, 면티셔츠, 무지티, MU-T, 홈웨어컬렉션]","14,900원",4.700718


# 사용자 행동 데이터 사용 ( SGD[사용자기반] / ALS )

- 점수에 대해 아이템과 사용자id로만 이루어진 데이터프레임 생성 

In [104]:
lower1=pd.read_excel(r'C:\Users\PC\프로젝트\사용자기반데이터\user_based_반팔left.xlsx')
lower2=pd.read_excel(r'C:\Users\PC\프로젝트\사용자기반데이터\user_based_홍균.xlsx')
lower3=pd.read_excel(r'C:\Users\PC\프로젝트\사용자기반데이터\사용자기반.하의(2).xlsx')
lower4=pd.read_excel(r'C:\Users\PC\프로젝트\사용자기반데이터\사용자기반하의왼쪽.xlsx')

df=pd.concat([lower1,lower2,lower3,lower4]).reset_index()
df.review=df.review.str.replace('\n','')
df=df.replace('Null',np.nan)
df.reset_index(inplace=True)
df.drop(['level_0','index'],axis=1,inplace=True)

df=df.drop_duplicates(['review'])
df.reset_index(inplace=True)
df.drop(['index'],axis=1,inplace=True)

cm=[]
for i in range(df.shape[0]):
    if re.findall(r'\d+cm',str(df['user_info'].iloc[i])) !=[]:
        cm.append(re.findall(r'\d+cm',df['user_info'].iloc[i]))
    else:
        cm.append(['Null'])
df=df.copy()        
df['cm']=cm

#몸무게추가
kg=[]
for i in range(df.shape[0]):
    if re.findall(r'\d+kg',str(df['user_info'].iloc[i])) !=[]:
        kg.append(re.findall(r'\d+kg',df['user_info'].iloc[i]))
    else:
        kg.append(['Null'])
df=df.copy()        
df['kg']=kg

#size추가
user_size=[]
for i in range(df.shape[0]):
    if re.findall(r'[A-Z]+',str(df['user_info'].iloc[i])) !=[]:
        user_size.append(re.findall(r'[A-Z]+',df['user_info'].iloc[i]))
    else:
        user_size.append(['Null'])
df=df.copy()        
df['user_size']=user_size

#리스트에 담겨 있어서 꺼내기
c=[];d=[];e=[]
for i in range(df.shape[0]):
    c.append(df['cm'][i][0])
    d.append(df['kg'][i][0])
    e.append(df['user_size'][i][0])
    
df['cm']=c
df.cm=df.cm.str.replace('cm','')
df['kg']=d
df.kg=df.kg.str.replace('kg','')
df['user_size']=e

df=df.replace('Null',np.nan)
df = df.astype({'cm': 'float'})
df = df.astype({'kg': 'float'})
df['bmi']=df.kg/(df.cm*df.cm)*10000


# 클렌징
df['tokens']=df.review.str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
idx=df[df.tokens==""].index
a=df.copy()
df=df.drop(idx)
df=df.dropna(subset=['tokens']) # 토큰화 결과 빈값과 null 셀 제거 

df.reset_index(inplace=True)

sent=pd.read_excel(r'C:\Users\PC\프로젝트\sent.xlsx')
sent.fillna(0,inplace=True)
sent_df=pd.concat([df,sent],axis=1)
sent_df['new_score']=sent_df.score+sent_df.result

In [105]:
df=sent_df.drop(['index','review','cm','kg','user_size','tokens','negative','neutral','positive'],axis=1)
df

,item_name,user_id,user_info,score,bmi,result,new_score
0,릴렉스 핏 크루 넥 반팔 티셔츠 [화이트],LV 4 봄이22222,"XL 구매 남성, 178cm, 66kg",10,20.830703,0.750000,10.750000
1,릴렉스 핏 크루 넥 반팔 티셔츠 [화이트],LV 4 성진짱123,M 구매,10,NaN,0.000000,10.000000
2,릴렉스 핏 크루 넥 반팔 티셔츠 [화이트],LV 2 뉴비_a0013bf6,"L 구매 남성, 178cm, 90kg",10,28.405504,0.521739,10.521739
3,릴렉스 핏 크루 넥 반팔 티셔츠 [화이트],LV 3 뉴비_423608cc,"XL 구매 남성, 178cm, 73kg",10,23.040020,-0.500000,9.500000
4,릴렉스 핏 크루 넥 반팔 티셔츠 [화이트],LV 3 쏘쏘_,L 구매,10,NaN,0.000000,10.000000
...,...,...,...,...,...,...,...
95217,[패키지] essential 스트링 세미 와이드 팬츠,LV 3 선녀와튀었꾼,"블랙 / M , 멜란지 , M 구매 남성, 174cm, 67kg",10,22.129740,1.000000,11.000000
95218,[패키지] essential 스트링 세미 와이드 팬츠,LV 3 핑플,"멜란지 / L , 블랙 , L 구매 남성, 170cm, 90kg",10,31.141869,0.635963,10.635963
95219,[패키지] essential 스트링 세미 와이드 팬츠,LV 3 정정휘휘찬찬,"블랙 / M , 멜란지 , M 구매 남성, 170cm, 54kg",10,18.685121,0.000000,10.000000
95220,[패키지] essential 스트링 세미 와이드 팬츠,LV 4 nowgnoj,"블랙 / M , 멜란지 , L 구매 남성, 173cm, 64kg",10,21.383942,1.000000,11.000000


In [106]:
df[df.item_name=='']

,item_name,user_id,user_info,score,bmi,result,new_score


In [107]:
# 피벗테이블 만들기
user_item_rating = pd.pivot_table(df,index='user_id',columns='item_name',values='new_score')
user_item_rating

item_name,#0085 greysh blue crop jeans,#0186 Heritage black mannish crop fit,#0198 Sensitive -A standard fit,#0199 Sensitive -B standard fit,#0203 off-white straight fit,#0214 Moderation indigo straight fit,#0215 Melt black straight fit,#0220 Elaboration straight crop fit,#0223 Melt black straight crop fit,#0245 Black crude wide fit,...,헤비웨이트 오버사이즈 후디드 스웨트셔츠 [미디엄 그레이],헤비코튼 자수로고 나시 3 color,화란 세미오버 니트 라벤더,화란 세미오버 니트 블랙,화이트 중기장 레이어드 슬리브리스 티셔츠 (421142LY61),화이트라벨 오버롤 네이비,후디드 스웨트셔츠 [멜란지 그레이],휩코드 세미 와이드 히든 밴딩 슬랙스 [블랙],휩코드 와이드 히든 밴딩 슬랙스 [블랙],히든 버튼 드레스 셔츠 [화이트]
user_id,,,,,,,,,,,,,,,,,,,,,
LV 1 Aqym,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LV 1 Ehrlich,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LV 1 Yang30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LV 1 Yuha810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LV 1 filter7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LV 8 훈형,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LV 8 휘휘꾸,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LV 8 ｎｏ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 1. 잠재요인협업필터링(사용자기반)

In [109]:
def get_rmse(R, P, Q, non_zeros):
    error= 0
    full_pred_matrix = np.dot(P, Q.T)
    
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

def matrix_factorization(R,K, steps=200, learning_rate=0.01, r_lambda = 0.01):
    num_users, num_items = R.shape
    # p 와 q 매트릭스의 크기를 지정, 정규 분포를 가진 랜덤한 값 입력
    np.random.seed(1)
    P = np.random.normal(scale=1./K, size = (num_users,K))
    Q = np.random.normal(scale=1./K, size = (num_items,K))
   
    prev_rmse=10000
    break_count=0
    
    # R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트 객체에 저장
    non_zeros = [ ( i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0]
    
    # SGD 기법으로 P와 Q 매트릭스를 업데이트
    for step in range(steps):
        
        for i,j,r in non_zeros:
            
            # 실제 값 예측 값 차이인 오류
            eij = r - np.dot(P[i,:], Q[j,:].T)
            
            # 정규화를 반영한 SGD 업데이트 공식적용
            P[i,:] = P[i,:] + learning_rate * (eij * Q[j,:] - r_lambda * P[i,:])
            Q[j,:] = Q[j,:] + learning_rate * (eij * P[i,:] - r_lambda * Q[j,:])
            
        rmse = get_rmse(R, P, Q, non_zeros)
        if (step %10) == 0 :
            print('### iteration step: ', step, 'rmse:', rmse)
            
    return P,Q

def get_unbuy_clothes(DF,user_id):
    user_rat=DF.loc[user_id,:]
    already_buy=user_rat[user_rat>0].index.tolist()
    clothes_list=DF.columns.tolist()
    unbuy_list=[clothe for clothe in clothes_list if clothe not in already_buy]
    return unbuy_list

def recomm_clothes_by_user_id(pred_df,user_id,unbuy_list,top_n):
    recomm_clothes=pred_df.loc[user_id,unbuy_list].sort_values(ascending=False)[:top_n]
    return recomm_clothes

In [102]:
user_item_rating.shape

(72019, 1611)

In [110]:
user_item_rating = pd.pivot_table(df,index='user_id',columns='item_name',values='score')
P, Q = matrix_factorization(user_item_rating.values, K=50, steps=200, learning_rate=0.01,
                           r_lambda=0.01)
pred_matrix = np.dot(P, Q.T)
user_item_pred_matrix_SGD = pd.DataFrame(data= pred_matrix, index= user_item_rating.index,
                                  columns = user_item_rating.columns)

### iteration step:  0 rmse: 9.537626264194422
### iteration step:  10 rmse: 0.996455430685022
### iteration step:  20 rmse: 0.5113225640844369
### iteration step:  30 rmse: 0.298529453914515
### iteration step:  40 rmse: 0.12664012899302388
### iteration step:  50 rmse: 0.05950371110268898
### iteration step:  60 rmse: 0.040023536128869906
### iteration step:  70 rmse: 0.03419259007176139
### iteration step:  80 rmse: 0.03229690966470164
### iteration step:  90 rmse: 0.03144651228688716
### iteration step:  100 rmse: 0.03098413640754676
### iteration step:  110 rmse: 0.03063971809446243
### iteration step:  120 rmse: 0.030311359980917257
### iteration step:  130 rmse: 0.029975635706967763
### iteration step:  140 rmse: 0.029631874087010856
### iteration step:  150 rmse: 0.0292836031913881
### iteration step:  160 rmse: 0.02893428015747611
### iteration step:  170 rmse: 0.02858662377237156
### iteration step:  180 rmse: 0.02824266230606912
### iteration step:  190 rmse: 0.0279038769183

In [113]:
unbuy_list=get_unbuy_clothes(user_item_rating,'LV 1 Aqym')
pd.DataFrame(recomm_clothes_by_user_id(user_item_pred_matrix_SGD,'LV 1 Aqym',unbuy_list,top_n=10))

,LV 1 Aqym
item_name,
[테이퍼드핏]I CREAM CHIP,4.325545
M 머스트해브 BOS 팬츠 SJ - 네이비 / FQ7708,3.747423
릴렉스 핏 크루 넥 포켓 반팔 티셔츠 [블랙],3.735042
[세트] 오버핏 기모 하프 집업 패키지,3.685002
라인 크롭 팬츠 (CP0090),3.560361
유니섹스 레잇 체크 셔츠-베이지,3.339260
소프트 코튼 크루넥 니트 빈티지브라운,3.316401
(유니섹스)M Royal Cargo-Jogger Pants(BLACK),3.300092
테이퍼드 크롭 밴딩 크림진 솔티데님.02,3.240879


## 2. ALS

In [115]:
user_item_rating = df.pivot_table("new_score",index = "user_id",columns = "item_name")
user_item_rating.fillna(0,inplace=True)
new_user=np.array(user_item_rating)
rating_matrix_user = scipy.sparse.csr_matrix(new_user)

In [116]:
als_model = ALS(factors=50, regularization=0.01, iterations = 300)
als_model.fit(rating_matrix_user.T)

  0%|          | 0/300 [00:00<?, ?it/s]

In [117]:
user_item_index= user_item_rating.rename_axis('index').reset_index()
item_name_index = user_item_rating.T.rename_axis('index').reset_index()

In [118]:
user_item_pred_matrix=np.dot(als_model.user_factors,als_model.item_factors.T) * 10
user_item_pred_matrix_ALS= pd.DataFrame(user_item_pred_matrix, index=user_item_index['index'],columns=item_name_index['index'])

In [119]:
user_item_pred_matrix_ALS

index,#0085 greysh blue crop jeans,#0186 Heritage black mannish crop fit,#0198 Sensitive -A standard fit,#0199 Sensitive -B standard fit,#0203 off-white straight fit,#0214 Moderation indigo straight fit,#0215 Melt black straight fit,#0220 Elaboration straight crop fit,#0223 Melt black straight crop fit,#0245 Black crude wide fit,...,헤비웨이트 오버사이즈 후디드 스웨트셔츠 [미디엄 그레이],헤비코튼 자수로고 나시 3 color,화란 세미오버 니트 라벤더,화란 세미오버 니트 블랙,화이트 중기장 레이어드 슬리브리스 티셔츠 (421142LY61),화이트라벨 오버롤 네이비,후디드 스웨트셔츠 [멜란지 그레이],휩코드 세미 와이드 히든 밴딩 슬랙스 [블랙],휩코드 와이드 히든 밴딩 슬랙스 [블랙],히든 버튼 드레스 셔츠 [화이트]
index,,,,,,,,,,,,,,,,,,,,,
LV 1 Aqym,0.434954,0.128479,0.111311,0.083670,0.292129,0.307083,0.561295,-1.067680,0.441323,0.327242,...,-0.249561,-0.005298,0.001459,0.251009,-0.034750,-0.042412,0.236945,0.298136,-0.051788,0.058935
LV 1 Ehrlich,-0.255924,0.128614,0.619993,-0.378859,-0.098193,-0.293653,-0.355443,0.219935,0.212163,-0.078842,...,0.329555,0.003416,-0.000853,0.071995,0.008554,0.014086,0.218984,0.057778,-0.037784,0.220873
LV 1 Yang30,0.562003,-0.480402,-0.587221,0.301687,0.388049,-0.231275,0.174142,-0.105997,-0.254019,-0.035889,...,0.146928,0.034786,0.005632,0.046316,-0.007462,0.061377,0.673279,-0.322129,0.216224,0.657152
LV 1 Yuha810,-0.411156,-0.710697,0.020460,-0.325803,-0.296729,-0.334043,-0.329320,-0.928510,0.206614,0.800316,...,0.330867,0.003678,-0.000322,0.081734,-0.015552,-0.009228,0.035438,-0.152292,-0.050385,0.159421
LV 1 filter7,0.037311,0.434520,-0.551466,-0.227964,-0.504990,-0.348847,-0.529617,0.793939,-0.405550,0.141947,...,0.150778,-0.015597,-0.003462,0.874298,-0.020071,0.014198,0.202452,-0.150354,0.046619,0.033669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LV 8 훈형,-2.491545,1.590447,7.475671,-2.295072,3.298466,0.836909,0.244994,3.000252,2.383489,-1.367979,...,0.370050,-0.096160,-0.045474,-3.082366,0.161373,0.381568,3.638671,1.137215,0.530047,-0.075223
LV 8 휘휘꾸,-0.398813,1.197887,0.817119,0.013608,0.616516,-0.171720,0.198797,0.083054,0.029140,-0.575944,...,0.364135,-0.011637,-0.014100,-0.370573,0.122337,0.021123,-0.058226,-0.090712,0.277983,2.047950
LV 8 ｎｏ,1.031543,0.161046,0.915477,-0.331268,0.628987,0.207419,0.165369,0.027602,-0.651007,0.386421,...,-0.513554,0.030228,-0.005074,0.253231,-0.083001,0.017082,-0.427757,0.104528,-0.154998,0.320811


In [120]:
unbuy_list=get_unbuy_clothes(user_item_rating,'LV 1 Aqym')
pd.DataFrame(recomm_clothes_by_user_id(user_item_pred_matrix_ALS,'LV 1 Aqym',unbuy_list,top_n=10))

,LV 1 Aqym
index,
[쿨탠다드] 쿨탠다드 피케 폴로 셔츠 [블랙],2.332105
S600 파워블랜드 무지 맨투맨 크루넥,2.058268
오픈 칼라 폴로 셔츠 [블랙],1.933521
릴렉스 핏 피케 폴로 셔츠 [블랙],1.611430
[3-PACK]베이직 트래블러 피그먼트 스웨트셔츠,1.525447
캐시미어 블렌드 리브드 크루 넥 니트 [블랙],1.510219
[패키지] 리오 오버핏 벌룬 후드 6종 2Pack KJHD2304,1.508644
TRIPLE STRIPE KNIT,1.503766
뉴 스트레이트 데님 팬츠 [BLACK],1.472677


### 결과들에 대한 앙상블 시도...

- tf_sim
- w2v_sim
- user_item_pred_matrix_SGD
- user_item_pred_matrix_ALS

In [138]:
sim_sort

array([[   0,  161,  108, ..., 2077, 2075, 1562],
       [   1,   20,  204, ..., 2079, 2078, 1562],
       [   2,  195,   18, ..., 2085, 2084, 1562],
       ...,
       [3124, 3122, 3120, ..., 2077, 2076,    0],
       [3075, 3123, 3074, ..., 1904, 1903,    0],
       [3124, 3122, 3120, ..., 2077, 2076,    0]], dtype=int64)

In [137]:
sim_sort[0]

array([   0,  161,  108, ..., 2077, 2075, 1562], dtype=int64)

In [204]:
sim_sort=tf_sim.argsort()[:,::-1]
a=int(DF[DF.item_name=='릴렉스 핏 크루 넥 반팔 티셔츠 '].index.values)
A=pd.Series(DF.iloc[sim_sort[a],:].item_name).reset_index().reset_index().drop('index',axis=1)

In [190]:
sim_sort=w2v_sim.argsort()[:,::-1]
a=int(DF[DF.item_name=='릴렉스 핏 크루 넥 반팔 티셔츠 '].index.values)
B=pd.Series(DF.iloc[sim_sort[a],:].item_name).reset_index().reset_index().drop('index',axis=1)

In [191]:
C=pd.Series(user_item_pred_matrix_SGD.loc['LV 1 Aqym',:].sort_values(ascending=False).reset_index().item_name).reset_index()

In [196]:
D=pd.Series(user_item_pred_matrix_ALS.loc['LV 1 Aqym',:].sort_values(ascending=False).reset_index()['index']).reset_index()
D.rename(columns={'index':'item_name'},inplace=True)

In [206]:
sample=pd.concat([A,B,C,D],axis=0)

In [207]:
# 이거 하는거 까먹지 말기~
sample['item_name']=sample.item_name.str.replace(r'\[[^)]*\]', '')
sample['item_name']=sample.item_name.str.replace(r'\([^)]*\)', '')

In [215]:
sample['level_0']=sample['level_0']+1
sample

,level_0,item_name,index
0,1.0,릴렉스 핏 크루 넥 반팔 티셔츠,NaN
1,2.0,릴렉스 핏 크루 넥 포켓 반팔 티셔츠,NaN
2,3.0,크루 넥 7부 티셔츠,NaN
3,4.0,베이식 크루 넥 반팔 티셔츠,NaN
4,5.0,실켓 릴렉스 핏 크루 넥 반팔 티셔츠,NaN
...,...,...,...
1742,1743.0,웨인듀 노멀 데님셔츠_DL_E30,NaN
1743,1744.0,클래식 숏 3부 블랙,NaN
1744,1745.0,1301 무지 반팔티 블랙&화이트,NaN
1745,1746.0,1301 무지 반팔티 블랙&화이트,NaN


In [213]:
pd.DataFrame(sample.groupby('item_name')['level_0'].sum().sort_values(ascending=False))

,level_0
item_name,
,31013.0
라이트웨이트 테이퍼드 히든 밴딩 크롭 슬랙스,10596.0
113 EASY CARE LINEN SHIRTS,8842.0
와이드 데님 팬츠,8743.0
사이드 트랙 팬츠,8104.0
...,...
수피마 릴렉스드 크루 넥 반팔 티셔츠,26.0
제이 양기모후드 그레이,24.0
폰테 크루 넥 긴팔 티셔츠,22.0
